In [ ]:
from multiprocessing import Queue, Process
from cpu.utils.cpu_monitor_live import run_cpu_monitor, run_pipeline, run_spinners
from cpu.utils.monitor_simulation import run_simulation
import pandas as pd


In [ ]:
# create a system model
from cpu.systems import CPUSystem
cpu = CPUSystem("cpu")

In [ ]:
queue = Queue()

sim_process = Process(target=run_simulation, args=(queue,))
sim_process.start()

monitor_process = run_cpu_monitor(queue)
pipeline = [(run_spinners, (8, 60)), (run_spinners, (1, 60))]
run_pipeline(pipeline)

monitor_process.join()
sim_process.join()
# Receive simulation results from the queue
if not queue.empty():
    results = queue.get()
    df = pd.DataFrame(results)

    # Save from notebook (safe path)
    df.to_csv("data/real_time_measure.csv", index=False)
    print("Results saved!")
else:
    print("Queue was empty. No results to save.")


In [ ]:
#Differentiate the simulated and measured data
data_sim = pd.read_csv("data/real_time_measure.csv")
data_mes = pd.read_csv("data/real_time_measure.csv")
data_sim.rename(columns={'T_cpu_simulated': 'T_cpu'}, inplace=True)
data_mes.rename(columns={'T_cpu_measured': 'T_cpu'}, inplace=True)

In [ ]:
from cpu.utils.plot_recorders import plot_recorders
plot_recorders(
    {
        "operation": data_mes,
        # "simulation": data_sim,
    },
    [
        [("time", "T_cpu"),("time", "cpu.usage")], 
        [("T_cpu", "Fan_rpm_1"),("time", "Fan_rpm_1")]
    ],
    width=600,
    height=350
)